## The File for generating HTML tags

In [1]:
from wand.image import Image as wi
import os; from io import BytesIO; import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time
import re
import glob
from tesserocr import PyTessBaseAPI, RIL, iterate_level, PT, OEM

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "first app")

In [4]:
def pdf2folderimages (dir_path, file):
    newfolderpath = dir_path + file.split('.')[0]
    print(newfolderpath)
    try:
        new_dir_cmd = "mkdir "+ newfolderpath # Creating a new directory for the file 
        os.system(new_dir_cmd) # Running the OS Command in the file 
        sys_cmd = "convert -units PixelsPerInch -density 300 " + dir_path+file +" "+ newfolderpath+"\page.png" # Converting the files 
        os.system(sys_cmd)
        return newfolderpath
    except:
        return False

In [5]:
def sorted_nicely(l):
    """ Sort the given iterable in the way numerical decimal order rather than binary."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

### Page limit 
* Whether the page is greater than the required threshold limit

In [44]:
def pdf_page_limit(pdf_file_location, threshold_limit):
    """
    A Function responsible for the page based threshold returning whether to do it or not 
    """
    pdf = wi(filename = pdf_file_location, resolution=5 ,background = 'white')
    numPages = len(pdf.sequence)
    print(numPages) # Debugging
    if numPages > threshold_limit:
        return False
    else:
        return True

#### Testing Page limit on the PDF files

In [45]:
directory = os.sep.join(['.','datafiles',''])

In [47]:
pdf_files = glob.glob(directory+'*.pdf')
for pdf_file in pdf_files:
    print(pdf_file)
    state = pdf_page_limit(pdf_file,50)
    print(state)

.\datafiles\IVA_2016_TEST_CHART_10020_Redacted.pdf
9
True
.\datafiles\IVA_2016_TEST_CHART_10504_Redacted.pdf
58
False
.\datafiles\IVA_2016_TEST_CHART_12062_Redacted.pdf
95
False
.\datafiles\IVA_2016_TEST_CHART_12852_Redacted.pdf
72
False
.\datafiles\IVA_2016_TEST_CHART_13117_Redacted.pdf
7
True
.\datafiles\IVA_2016_TEST_CHART_15149_Redacted.pdf
20
True
.\datafiles\IVA_2016_TEST_CHART_15347_Redacted.pdf
18
True
.\datafiles\IVA_2016_TEST_CHART_15913_Redacted.pdf
5
True
.\datafiles\IVA_2016_TEST_CHART_169_Redacted.pdf
45
True
.\datafiles\IVA_2016_TEST_CHART_18125_Redacted.pdf
10
True
.\datafiles\IVA_2016_TEST_CHART_18251_Redacted.pdf
11
True
.\datafiles\IVA_2016_TEST_CHART_18530_Redacted.pdf
38
True
.\datafiles\IVA_2016_TEST_CHART_18548_Redacted.pdf
6
True
.\datafiles\IVA_2016_TEST_CHART_19706_Redacted.pdf
14
True
.\datafiles\IVA_2016_TEST_CHART_20143_Redacted.pdf
95
False
.\datafiles\IVA_2016_TEST_CHART_20148_Redacted.pdf
4
True
.\datafiles\IVA_2016_TEST_CHART_20160_Redacted.pdf
124
Fals

### Spark Session and Creating of Absolute based image paths
* Creating Image path mappings along with direct dataframes creationg

In [56]:
from pyspark.sql import SparkSession
import sys
import datetime

In [57]:
spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()

In [69]:
full_image_path = [ [directory +path] for path in list_pdf_files]

In [72]:
from pyspark.sql import Row
df=spark.createDataFrame(full_image_path,['image_paths'])

In [73]:
df.show()

+--------------------+
|         image_paths|
+--------------------+
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
|.\datafiles\IVA_2...|
+--------------------+
only showing top 20 rows



#### Please map the above configuration in a way

pdf_location_path_name, Image_location_path_name, ImageBinary, 

In [79]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pickle as pk

In [ ]:
def dummy_function(data_str):
         cleaned_str = 'dummyData'
         return data_str

In [ ]:
with open("complete_dict.pkl", 'rb') as f:
       dictionary = pk.load(f)

In [ ]:
pytestapi_path = r'C:\\Tesseract-OCR\\tessdata'

In [80]:
def get_words_dict_counts(tess_word):
    '''
    params:
    string text
    returns:
    words count
    words count, not in dictionary
    list of words in input strring
    '''
    words_count = 0
    wordsNotInDict = 0
    Words = ""
    
    lower_words = word_tokenizer(tess_word)
    for word in lower_words:
        words_count += 1
        Words += word + "\n"
        if word not in dictionary:  #assumption that dictionary is global variuable containing all dictionary words in alist  
            wordsNotInDict += 1
    
    return words_count, wordsNotInDict, Words

In [81]:
def word_tokenizer(text):
    tokens = re.findall("[a-z]+", text)
    return tokens

### Get Dirty Pages UDF Based

In [82]:
def get_dirt_from_page(filepath):
    
    results = {}
    
    tmpTotal_words = 0
    tmpNot_in_dict = 0
    tmpWords = ""
    page = Image.open(filepath) #readimage in PIL format
    print("MR chart '", filepath, "' is under-process")
    DICT_time = 0
    
    with PyTessBaseAPI(path = pytestapi_path) as api:
        api.SetImage(page)
        api.Recognize()
        ## Preprosessing for background detection --START
        ri = api.GetIterator()
        level = RIL.WORD
        
        # Word by word iterator 
        for r in iterate_level(ri, level):
            if r:
                s = time.time()
                tmp = r.GetUTF8Text(level)
                if (tmp.rstrip()):
                    w_c, wd_c, w = get_words_dict_counts(r.GetUTF8Text(level).lower())
                    tmpTotal_words += w_c
                    tmpNot_in_dict += wd_c
                    tmpWords += w
                DICT_time += time.time() - s
                
    print("Dictionary check time:\t", DICT_time)
    results['invalidWords'] = tmpNot_in_dict
    results['totalWords'] = tmpTotal_words
    results['tmpWords'] = tmpWords
    return results

### UDFs

In [83]:
s = time.time()
dummy_function_udf = udf(get_dirt_from_page, StringType())
df = df.withColumn("dirt_calculate", dummy_function_udf(df['image_paths']))
df.show()
print(time.time() - s)

Py4JJavaError: An error occurred while calling o70.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 253, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 248, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 331, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 140, in dump_stream
    for obj in iterator:
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 320, in _batched
    for item in iterator:
  File "<string>", line 1, in <lambda>
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 76, in <lambda>
  Fil

In [ ]:
print("Average Time:\t\t", tmp_time/129)

In [ ]:
pytestapi_path = "C:\\Tesseract\\Tesseract-OCR-v5\\tessdata"

### OCR SCAN
* Images to Text Conversion 
* Table Detection (Table detection does happens in the java code but doesn't go in the grain)
     * TH , TR , TD
* Background Detection
    * Specific Line Background
    * Word Level, Block Level & Line Level (Under Progress)
    
* Font Characterstics
    * Font Density ( Tried to do in jar but was inefficent, insufficent and inaccurate)
        * Bold 
        * Light
    * Font Size
* Hand Written Detection (ITS UNDER PROGRESS)
    * HandWritten Model vs Digital Text Model
    * Evaluation of Metrices
    
     
      

In [ ]:
def line_bg_detection(bbox, numBIN, NUMPYimage):
    if bbox:
        x1, y1, x2, y2 = bbox
        colorbin = np.ones((numBIN, 3), dtype = 'int16') *-1

        combs = np.where(NUMPYimage[y1:y2, x1:x2, 0] != -1) # Check if the image didn't had the -1 there
        if (combs[0].size != 0): # check if the size is not zero 
            bin_i = 0
            # Choose randomly bins (20 is the default set right now )
            # To check the image or the bin or whatsoever 
            for i in np.random.choice(range(combs[0].shape[0]), numBIN):
                x = combs[1][i]
                y = combs[0][i]
                colorbin[bin_i] = NUMPYimage[y1+y, x1+x, :]
                bin_i += 1
            # Historgram 
            hist = {}
            for r, g, b in colorbin:
                if not (r == -1):
                    RGB = str(r)+"_"+str(g)+"_"+str(b)
                    if RGB in list(hist.keys()):
                        hist[RGB] += 1
                    else:
                        hist[RGB] = 1
            if len(list(hist.keys())):
                return list(hist.keys())[list(hist.values()).index(max(list(hist.values())))].split("_") ## RGB value in list
    return 0

In [ ]:
def get_table_row_tags(api, bbox):
    ri = api.GetIterator()
    level = RIL.TEXTLINE
    ri = api.GetIterator()
    
    buffer = 0
    
    lines = "<table>\n"
    
    for rl in iterate_level(ri, level):
        if rl:
            tmp = rl.GetUTF8Text(level)
            if (tmp.rstrip()):
                c_x1, c_y1, c_x2, c_y2  = rl.BoundingBox(level)
                if (c_y1 >= (bbox[1] - buffer) and c_y2 <= (bbox[3] + buffer)):
                    lines += "<tr>" + rl.GetUTF8Text(level) + "\n</tr>\n"
                if (c_y2 > (bbox[3] + buffer)):
                    break
    lines += "</table>\n"
    # print(lines)
    return lines

In [ ]:
def sorted_nicely(l):
    """ Sort the given iterable in the way that humans expect."""
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(l, key = alphanum_key)

In [ ]:
def generate_HTML (FOLDERpath):
    '''
    params:
    FOLDERpath --> path of directoray which contain all the MR document images
    returns:
    HTML format string containing result of OCR scan    
    '''
    
    FileTextBuffer = ""
    FileTextBuffer += "<document>\n"

    filesList = os.listdir(FOLDERpath)
    
    pagecounter  = 1
    
    WordsofPages = FileTextBuffer
    for filename in sorted_nicely(filesList):
        WordsofPages += "<page"+str(pagecounter)+">\n"
        
        start = time.time()
        page = Image.open(FOLDERpath+"\\"+filename) #readimage in PIL format
        print("MR chart '", FOLDERpath+"\\"+filename, "' is under-process")
        
        thisPageText = ""
        thisPageText += "<page" +str(pagecounter)+">\n"

        with PyTessBaseAPI(path = pytestapi_path) as api:
            api.SetImage(page)
            api.Recognize()
            
            if (page.mode != 'RGB'): #check if image not in RGB then make it one: for bg color detection
                NUMPYimage = np.array(page.convert('RGB'), dtype='int16')
            else:
                NUMPYimage = np.array(page, dtype='int16')

            ## Preprosessing for background detection --START
            ri = api.GetIterator()
            level = RIL.WORD
            # Word by word iterator 
            for r in iterate_level(ri, level):
                if r:
                    bbox = r.BoundingBox(level)
                    WordsofPages += r.GetUTF8Text(level).lower() + "\n"
                    if bbox: ## Now black (-1) 'em out
                        NUMPYimage[bbox[1]:bbox[3], bbox[0]:bbox[2], :] = -1
            ## Preprosessing for background detection --END

            ri = api.GetIterator()
            level = RIL.BLOCK # Block based Values 

            # Table is detectable ( we can get the verticle and horizental Lins )
            for r in iterate_level(ri, level):
                if r:
                    block_type = r.BlockType() # Type of that specific block
                    # print(r.GetUTF8Text(level))
                    
                    if (block_type == PT.TABLE):
                        img = r.GetBinaryImage(level)
                        # img.show() to show cropped table image
                        text_rows = get_table_row_tags(api, r.BoundingBox(level))
                        thisPageText += text_rows
                    elif(block_type not in [PT.UNKNOWN, PT.FLOWING_IMAGE, PT.HEADING_IMAGE, PT.PULLOUT_IMAGE, PT.HORZ_LINE, PT.VERT_LINE, PT.NOISE]):
                        ###  Backgrpound Color algo -START Params (bbox, numBIN, NumpyImage)
                        rgb_result = line_bg_detection(r.BoundingBox(level), 20, NUMPYimage)
                        ###  Backgrpound Color algo -END
                        if (rgb_result):
                            thisPageText += "<text font_bg = '" + rgb_result[0] + "," + rgb_result[1] + "," + rgb_result[2] +"'>\n" + r.GetUTF8Text(level) + "</text>\n"
                        else:
                            thisPageText += "<text>\n" + r.GetUTF8Text(level) + "</text>\n"

        FileTextBuffer += thisPageText + "</page" +str(pagecounter)+">\n"
        WordsofPages += "<\page"+str(pagecounter)+">\n"
        
        pagecounter += 1
        print("Page Time:\t\t", time.time() - start)
        
    FileTextBuffer += "</document>\n"
    WordsofPages += "</document>\n"
    
    return FileTextBuffer, WordsofPages

In [ ]:
FOLDERpath = '..\\datafiles\\doc22'

In [ ]:
COST = time.time()
GeneratedText, GeneratedWords = generate_HTML(FOLDERpath)
tmp_time = time.time() - COST
print("Time on Document:\t", tmp_time)

In [ ]:
print("Average time:\t\t", tmp_time/52)

In [ ]:
print(GeneratedText)

In [ ]:
with open('doc22-text8.xml', 'w+') as f:
    f.write(GeneratedText)

In [ ]:
with open('doc22-words-2.xml', 'w+') as f:
    f.write(GeneratedWords)